# Getting Your Data From Yelp!

In order to make sure you are on track to completing the project, you will complete this workbook first. Below are steps that you need to take in order to make sure you have your data from yelp and are ready to analyze it. Your cohort lead will review this workbook with you the Wednesday before your project is due.    

## Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The api, however, provides you with a lot of information that will not be pertinent to your analysis. YOu will pull data from the api and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the api will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

___

## Part 2 - Create ETL pipeline for the business data from the API

Now that you know what data you need from the API, you want to write code that will execute a api call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*api call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

- Write a function to make a call to the yelp API

In [83]:
import pandas as pd
import requests
import json
url = 'https://api.yelp.com/v3/businesses/search'
api_key = "S3cROVIGs17gTiCO6iy8-2ycbL999zhz84ANxStc2mYNESu0cKfM4qdarKNZ69t-2bnTPSR_RRTTCTKmpQidQYhD1mpVN1zvyo0EGTVthI7SVumcOA3IZrRgj5omYHYx"
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
# what type of business do you want to search
term = 'Thai'
term2 = 'Mexican'
#where do you want to perform this search
location = 'Queens'

# what is your other parameter you want to search against
categories = 'restaurants'
url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
                "offset": None
    
            }
url_params2 = {
                "term": term2.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "categories" : categories,
                "limit": 50,
    
            }

In [108]:

def yelp_call(url_params):
    
    response = requests.get(url, headers=headers, params=url_params)
    return response.json()['businesses']
    

In [109]:
thai_data = yelp_call(url_params)

In [110]:
mex_data = yelp_call(url_params2)

In [111]:
thai_data

[{'id': 'ubX4CD4GqXpXjLfuBJo8HQ',
  'alias': 'spicy-republic-new-york',
  'name': 'Spicy Republic',
  'image_url': '',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/spicy-republic-new-york?adjust_creative=YYinute8YoprX7XHYr6zhg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=YYinute8YoprX7XHYr6zhg',
  'review_count': 2,
  'categories': [{'alias': 'thai', 'title': 'Thai'}],
  'rating': 2.5,
  'coordinates': {'latitude': 40.7557, 'longitude': -73.96771},
  'transactions': [],
  'price': '$',
  'location': {'address1': '972 Second Ave',
   'address2': '',
   'address3': '',
   'city': 'New York',
   'zip_code': '10021',
   'country': 'US',
   'state': 'NY',
   'display_address': ['972 Second Ave', 'New York, NY 10021']},
  'phone': '+12124212324',
  'display_phone': '(212) 421-2324',
  'distance': 10034.630240390272},
 {'id': '9DceMTrjyVqBgQRqgR6GWw',
  'alias': 'red-bamboo-new-york',
  'name': 'Red Bamboo',
  'image_url': 'https://s3-media4.fl.yelpcdn.com

- Write a function to parse the API response so that you can easily insert the data in to the DB

In [112]:
def parse_results(list_of_data):
    # create a container to hold our parsed data
    biz_list = []
    # loop through our business and 
    for business in list_of_data:
    # parse each individual business into a tuple
        try:
            biz_tuple = (business['id'],
                     business['name'],
                     business['location']['address1'],
                     business['rating'],
                     business['price'],
                     business['location']['zip_code'],
                     business['transactions'],
                     business['review_count'],
                     business['categories'])   
    # add each individual business tuple to our data container
            biz_list.append(biz_tuple)
        except:
            continue
    # return the container with all of the parsed results
    return biz_list

In [114]:
parsed_thai = parse_results(thai_data)

In [115]:
parsed_mex = parse_results(mex_data)

In [116]:
parsed_mex

[('Yilk_Ucz4L6w9EmvAS2gHg',
  'Taqueria Coatzingo',
  '7605 Roosevelt Ave',
  4.0,
  '$$',
  '11372',
  ['delivery', 'pickup'],
  440,
  [{'alias': 'mexican', 'title': 'Mexican'}]),
 ('j4dGkqL-OK9YH8Sl2eOWEg',
  '5 Burro Cafe',
  '72-05 Austin St',
  3.5,
  '$$',
  '11375',
  ['pickup', 'delivery'],
  1398,
  [{'alias': 'mexican', 'title': 'Mexican'},
   {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}]),
 ('-kVueEw_fyO1Q2Gkmj6Few',
  'Taqueria Santa Fe',
  '4708 47th Ave',
  4.5,
  '$',
  '11377',
  ['delivery', 'pickup'],
  380,
  [{'alias': 'mexican', 'title': 'Mexican'}]),
 ('MhGjWgAFFyb6QMwUIkhpBQ',
  'Metro Taco',
  '102-15 Metropolitan Ave',
  4.0,
  '$$',
  '11375',
  ['delivery', 'pickup'],
  222,
  [{'alias': 'tacos', 'title': 'Tacos'},
   {'alias': 'bars', 'title': 'Bars'},
   {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}]),
 ('bNPYPLuCjnRvRuL56otJiw',
  'Don Tequila Urban Cantina',
  '83-38 Woodhaven Blvd',
  4.0,
  '$$',
  '11385',
  ['delivery', 'pickup'

- Write a function to take your parsed data and add it to the csv file where you will store all of your results. 

In [117]:
thai_df = pd.DataFrame(parsed_thai, columns = ['Id','Name', 'Location', 'Rating', 'Price', 'Zipcode', 'Transactions', 'Review Count', 'Categories'])

In [64]:
mex_df = pd.DataFrame(parsed_mex, columns = ['Name', 'Location', 'Rating', 'Price', 'Zipcode', 'Transactions', 'Review Count', 'Categories'])

In [118]:
thai_df

,Id,Name,Location,Rating,Price,Zipcode,Transactions,Review Count,Categories
0,ubX4CD4GqXpXjLfuBJo8HQ,Spicy Republic,972 Second Ave,2.5,$,10021,[],2,"[{'alias': 'thai', 'title': 'Thai'}]"
1,9DceMTrjyVqBgQRqgR6GWw,Red Bamboo,140 W 4th St,4.0,$$,10012,"[delivery, pickup]",1712,"[{'alias': 'vegetarian', 'title': 'Vegetarian'..."
2,HvN9eVtlm9MnOQCL_GlRSg,Pho Saigon,987 1st Ave,3.5,$$,10022,"[delivery, pickup]",179,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]"
3,irRF-5aowSiEycwkKGw3Jw,Pho Today - East Rutherford,75 NJ-17,4.0,$$,07073,[delivery],154,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]"
4,Ckz_1t4nUoFf2Yw-uuK4jQ,Mr Chen,5905 Myrtle Ave,3.0,$,11385,"[delivery, pickup]",69,"[{'alias': 'chinese', 'title': 'Chinese'}]"
5,ZLZswviJ0CSX5ZeQFSO2sQ,Pho Vietnam,1243 Ave U,4.0,$,11229,"[delivery, pickup]",307,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]"
6,tS4aiWBYgzlqKX898jr3_w,Shun Wang Restaurant,81-25 Broadway,3.5,$,11373,[delivery],171,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia..."
7,V2P3CWfUW_olSywlOlHGjQ,Asian Fusion,7827 Bergenline Ave,4.5,$$,07047,"[delivery, pickup]",99,"[{'alias': 'asianfusion', 'title': 'Asian Fusi..."
8,zv3QIA-xGfegH9EGpxgriQ,Tasty & Co,94-19 63rd Dr,3.5,$,11374,"[delivery, pickup]",79,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {..."
9,Bb1tUdH05NgOH8MCqAMvAQ,Bachi 197,197 Hackensack St,4.0,$,07075,"[delivery, pickup]",129,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a..."


In [36]:
# csv = business_df.to_csv('business_csv')

In [40]:
pd.read_csv('business_csv')

,Unnamed: 0,Name,Location,Rating,Price,Zipcode,Transactions,Review Count,Categories
0,0,Sup Thai Kitchen,178-19 Union Tpke,4.5,$$,11366,"['pickup', 'delivery']",373,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':..."
1,1,Kin'd,192-03 Union Tpke,4.5,$$,11366,"['pickup', 'delivery']",767,"[{'alias': 'thai', 'title': 'Thai'}]"
2,2,SriPraPhai Thai Restaurant,64-13 39th Ave,4.0,$$,11377,[],2380,"[{'alias': 'thai', 'title': 'Thai'}]"
3,3,Kitchen 79,37-70 79th St,4.5,$$,11372,"['pickup', 'delivery']",629,"[{'alias': 'thai', 'title': 'Thai'}]"
4,4,Ayada Thai,7708 Woodside Ave,4.0,$$,11373,"['pickup', 'delivery']",1222,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':..."
5,5,BKNY Thai Restaurant,47-11 Francis Lewis Blvd,4.0,$$,11361,['delivery'],1317,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':..."
6,6,Bangkok Cuisine,10718 70th Rd,4.0,$$,11375,"['pickup', 'delivery']",421,"[{'alias': 'thai', 'title': 'Thai'}]"
7,7,Pye Boat Noodle,35-13 Broadway,4.5,$$,11106,['delivery'],837,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':..."
8,8,Boon Chu Thai Place,83-18 Broadway,4.0,$,11373,"['pickup', 'delivery']",313,"[{'alias': 'thai', 'title': 'Thai'}]"
9,9,Bangkok Avenue,253-21 Northern Blvd,5.0,$$,11362,"['pickup', 'delivery']",86,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':..."


In [136]:
def df_save(csv_file_path, parsed_results):
    # your code to open the csv file, concat the current data, and save the data. 
    business_df = pd.DataFrame(parsed_results, columns = ['Id','Name', 'Location', 'Rating', 'Price', 'Zipcode', 'Transactions', 'Review Count', 'Categories'])
    business_df.to_csv(csv_file_path, mode = 'a')
    new_df = pd.read_csv(csv_file_path, delimiter = ",")
    return new_df
#     pd.DataFrame(parsed_results).to_csv()

In [124]:
thai_csv = df_save('thai_csv', parsed_thai)

In [135]:
mex_csv = df_save('mex_csv', parsed_mex)

In [125]:
thai_csv

,Unnamed: 0,Id,Name,Location,Rating,Price,Zipcode,Transactions,Review Count,Categories
0,0,ubX4CD4GqXpXjLfuBJo8HQ,Spicy Republic,972 Second Ave,2.5,$,10021,[],2,"[{'alias': 'thai', 'title': 'Thai'}]"
1,1,9DceMTrjyVqBgQRqgR6GWw,Red Bamboo,140 W 4th St,4.0,$$,10012,"['delivery', 'pickup']",1712,"[{'alias': 'vegetarian', 'title': 'Vegetarian'..."
2,2,HvN9eVtlm9MnOQCL_GlRSg,Pho Saigon,987 1st Ave,3.5,$$,10022,"['delivery', 'pickup']",179,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]"
3,3,irRF-5aowSiEycwkKGw3Jw,Pho Today - East Rutherford,75 NJ-17,4.0,$$,7073,['delivery'],154,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]"
4,4,Ckz_1t4nUoFf2Yw-uuK4jQ,Mr Chen,5905 Myrtle Ave,3.0,$,11385,"['delivery', 'pickup']",69,"[{'alias': 'chinese', 'title': 'Chinese'}]"
5,5,ZLZswviJ0CSX5ZeQFSO2sQ,Pho Vietnam,1243 Ave U,4.0,$,11229,"['delivery', 'pickup']",307,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]"
6,6,tS4aiWBYgzlqKX898jr3_w,Shun Wang Restaurant,81-25 Broadway,3.5,$,11373,['delivery'],171,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia..."
7,7,V2P3CWfUW_olSywlOlHGjQ,Asian Fusion,7827 Bergenline Ave,4.5,$$,7047,"['delivery', 'pickup']",99,"[{'alias': 'asianfusion', 'title': 'Asian Fusi..."
8,8,zv3QIA-xGfegH9EGpxgriQ,Tasty & Co,94-19 63rd Dr,3.5,$,11374,"['delivery', 'pickup']",79,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {..."
9,9,Bb1tUdH05NgOH8MCqAMvAQ,Bachi 197,197 Hackensack St,4.0,$,7075,"['delivery', 'pickup']",129,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a..."


- Write a script that combines the three functions above into a single process.

While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

In [74]:
from helper_funcs import *

ModuleNotFoundError: No module named 'helper_funcs'

In [137]:
###THAI###

# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params)
    results_mex=yelp_call(url_params2)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    parsed_results_mex=parse_results(results_mex)
    # use your function to insert your parsed results into the db
    df_save('thai_csv', parsed_results)
    df_save('mex_csv', parsed_results_mex)
    #increment the counter by 50 to move on to the next results
    cur += 50

In [142]:
thaidf = pd.read_csv('thai_csv')

In [143]:
mexdf = pd.read_csv('mex_csv')

___

## Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you ahve a model of the functions you will need to write, and how to pull them together in one script. For this part, you have the process below 

- In order to pull the reveiws, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

In [144]:
thaidf

,Unnamed: 0,Id,Name,Location,Rating,Price,Zipcode,Transactions,Review Count,Categories
0,0.0,ubX4CD4GqXpXjLfuBJo8HQ,Spicy Republic,972 Second Ave,2.5,$,10021,[],2,"[{'alias': 'thai', 'title': 'Thai'}]"
1,1.0,9DceMTrjyVqBgQRqgR6GWw,Red Bamboo,140 W 4th St,4.0,$$,10012,"['delivery', 'pickup']",1712,"[{'alias': 'vegetarian', 'title': 'Vegetarian'..."
2,2.0,HvN9eVtlm9MnOQCL_GlRSg,Pho Saigon,987 1st Ave,3.5,$$,10022,"['delivery', 'pickup']",179,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]"
3,3.0,irRF-5aowSiEycwkKGw3Jw,Pho Today - East Rutherford,75 NJ-17,4.0,$$,07073,['delivery'],154,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]"
4,4.0,Ckz_1t4nUoFf2Yw-uuK4jQ,Mr Chen,5905 Myrtle Ave,3.0,$,11385,"['delivery', 'pickup']",69,"[{'alias': 'chinese', 'title': 'Chinese'}]"
...,...,...,...,...,...,...,...,...,...,...
1708,39.0,z4648FlI4pl4vgK-koe3Lg,Komo Asian Cuisine,221 Bedford Ave,4.5,$$,11710,"['delivery', 'pickup']",102,"[{'alias': 'asianfusion', 'title': 'Asian Fusi..."
1709,40.0,JHCf1edqV2CyXBP5hu53Jg,Tokyo Cafe,59 Washington Ave,3.5,$$,07675,['delivery'],42,"[{'alias': 'japanese', 'title': 'Japanese'}, {..."
1710,41.0,BEuZGvNlbM4tXq7YICyM6w,LuAnne's Wild Ginger All-Asian Vegan,380 Broome St,4.0,$$,10013,"['delivery', 'pickup']",649,"[{'alias': 'asianfusion', 'title': 'Asian Fusi..."
1711,42.0,nxQW-FbIeYE4Jk7rKeiezA,Bambou,328 Pemberwick Rd,4.0,$$$,06831,"['delivery', 'pickup']",70,"[{'alias': 'asianfusion', 'title': 'Asian Fusi..."


In [150]:
business_id = list(thaidf['Id']) +list(mexdf['Id'])

In [175]:
business_id

['ubX4CD4GqXpXjLfuBJo8HQ',
 '9DceMTrjyVqBgQRqgR6GWw',
 'HvN9eVtlm9MnOQCL_GlRSg',
 'irRF-5aowSiEycwkKGw3Jw',
 'Ckz_1t4nUoFf2Yw-uuK4jQ',
 'ZLZswviJ0CSX5ZeQFSO2sQ',
 'tS4aiWBYgzlqKX898jr3_w',
 'V2P3CWfUW_olSywlOlHGjQ',
 'zv3QIA-xGfegH9EGpxgriQ',
 'Bb1tUdH05NgOH8MCqAMvAQ',
 'ojbH3wnRu050hRhkmoxRiA',
 'd8zSYpArh4oXpxeJWNA4Pg',
 'OZw7uy4wQyOUNF6gYC56sg',
 '-6KT1D508e6zvqHJvv4r2Q',
 'CMm3Xf-9v3QJ4ge20MoEVg',
 '0bDtHB75wr16A_8zUm_2ng',
 'GzY2EjCmGNgvRd7IE20fTA',
 'KQhsU3i6CIrwzoOxPlThng',
 'tu-uJ3mErdOkFo-p4Box6A',
 'G77Fo795EhpO48Lst9eo3A',
 'No4ny0agfERlycMQwkLD4w',
 'J5YMEzkPrDFTCIFIfK-3bg',
 'mc9lvusD4Na-KXT2z4ulzQ',
 'IoAUbSudW1Mds5_XY_01Rg',
 'mhIxjP-wjfihK8BTuf-Apw',
 'EIcbGkl6bRtAi12zcjA7-A',
 '8M_0cxAChoXl7Ls9YkWJlg',
 'YES1oCXuC-uggcnr8txElw',
 'QGldR7s9_qV_35iGtkcNyA',
 'iiZpFWVd1Cmd-cSOanmXAA',
 'bH6U8bze3zVTUBOPm913NQ',
 'ptWTdgSzSpH-0LD24QIH-w',
 'PYN67roncq-uYxKWPd7ukQ',
 'LjBKkUSpMwrpo99hXgtHYw',
 '71EQXj4urSJKrM7bZwM4Fg',
 '5RjZCsXuWPclyrDsnsXw7w',
 'ByTPZl9dL1wgiXEwChnowg',
 

In [191]:
# url_rev = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(ID)
api_key = "S3cROVIGs17gTiCO6iy8-2ycbL999zhz84ANxStc2mYNESu0cKfM4qdarKNZ69t-2bnTPSR_RRTTCTKmpQidQYhD1mpVN1zvyo0EGTVthI7SVumcOA3IZrRgj5omYHYx"
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }
# what type of business do you want to search

#where do you want to perform this search
# locale = 'Queens'

# what is your other parameter you want to search against
categories = 'restaurants'
url_params_rev = {"limit": 50}

In [192]:
urls =[]
for x in business_id:
    url_rev = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(x)
    urls.append(url_rev)

In [214]:
smaller = urls[0:20]

In [215]:
def yelp_call_rev(url_params):
    review_list =[]
    while len(review_list) < 2:
        for u in smaller:
        
            response = requests.get(u, headers=headers, params=url_params_rev)
            review_list.append(response.json())
    return review_list
    

In [201]:
print('hi')

hi


In [216]:
yelp_call_rev(url_params)

[{'reviews': [{'id': 'TuOgKNeY-Rz_OGLZhmVSXg',
    'url': 'https://www.yelp.com/biz/spicy-republic-new-york?adjust_creative=YYinute8YoprX7XHYr6zhg&hrid=TuOgKNeY-Rz_OGLZhmVSXg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=YYinute8YoprX7XHYr6zhg',
    'text': "Meh.  Color me unimpressed.\n\nMy office ordered lunch from there yesterday, and I got Pad Thai.  Not that it's my favorite or anything, but I feel it's a...",
    'rating': 2,
    'time_created': '2008-09-06 10:38:40',
    'user': {'id': 'aIA3Xo5YH4cg7MlHMhgxAg',
     'profile_url': 'https://www.yelp.com/user_details?userid=aIA3Xo5YH4cg7MlHMhgxAg',
     'image_url': 'https://s3-media4.fl.yelpcdn.com/photo/ZZ7kdOCWL614kIm8OLSN0g/o.jpg',
     'name': 'Adam R.'}},
   {'id': '_DkTpla00SQ-CD-BULeQWw',
    'url': 'https://www.yelp.com/biz/spicy-republic-new-york?adjust_creative=YYinute8YoprX7XHYr6zhg&hrid=_DkTpla00SQ-CD-BULeQWw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=YYinute8YoprX7

- Write a function that takes a business id and makes a call to the API for reivews


In [169]:
review_list = yelp_call_rev(url_params_rev)

- Write a function to parse out the relevant information from the reviews

In [168]:
def parse_results_review(list_of_data):
    # create a container to hold our parsed data
    review_list = []
    # loop through our business and 
    for review in list_of_data:
    # parse each individual business into a tuple
        try:
            review_tuple = (review['id'],
                     review['text'],
                     review['rating'],
                     review['time_created'],
                    )   
    # add each individual business tuple to our data container
            review_list.append(review_tuple)
        except:
            continue
    # return the container with all of the parsed results
    return review_list

In [171]:
parsed_reviews = parse_results_review(review_list)

- Write a function to save the parse data into a csv file containing all of the reviews. 

In [172]:
def df_save_review(csv_file_path, parsed_results):
    # your code to open the csv file, concat the current data, and save the data. 
    business_df = pd.DataFrame(parsed_results, columns = ['ID', 'Review', 'Rating', 'Date'])
    business_df.to_csv(csv_file_path, mode = 'a')
    new_df = pd.read_csv(csv_file_path, delimiter = ",")
    return new_df

In [174]:
reviewdf = df_save_review('review_csv', parsed_reviews)

- Combine the functions above into a single script  

In [ ]:
# create a variable  to keep track of which result you are in. 


#set up a while loop to go through and grab the result 
while cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params)
    results_mex=yelp_call(url_params2)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    parsed_results_mex=parse_results(results_mex)
    # use your function to insert your parsed results into the db
    df_save('thai_csv', parsed_results)
    df_save('mex_csv', parsed_results_mex)
    #increment the counter by 50 to move on to the next results
    cur += 50

___

## Part 4 -  Using python and pandas, write code to answer the questions below. 


- Which are the 5 most reviewed businesses in your dataset?
- What is the highest rating recieved in your data set and how many businesses have that rating?
- What percentage of businesses have a rating greater than or  4.5?
- What percentage of businesses have a rating less than 3?
- What percentage of your businesseshave a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?
- Return the text of the reviews for the most reviewed business. 
- Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 
- Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 


___

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20